# Collect reviews on the list of applications from the App Store and Google Store

**Correctness checked for:**
+ python 3.8.8
+ pandas 1.2.4
+ numpy 1.20.1

In [1]:
from app_store_scraper import AppStore
from google_play_scraper import Sort, reviews
import pandas as pd
import numpy as np

In [2]:
#read file with list of applications 
crypto_app = pd.read_csv('cryptocurrency-related applications.csv')
crypto_app.head()

,Name,AppStore_name,AppStore_id,GoogleStore_name
0,Crypto.com,crypto-com-buy-bitcoin-eth,1262148500,co.mona.android
1,Binance,binance-buy-bitcoin-crypto,1436799971,com.binance.dev
2,KuCoin,kucoin-buy-bitcoin-ether,1378956601,com.kubi.kucoin
3,Crypto Bubbles,crypto-bubbles,1599892658,net.cryptobubbles
4,OWNR Digital Wallet,ownr-digital-wallet,1459364947,com.ownrwallet.wallet


In [3]:
#collect 100 reviews for each app
df_all = pd.DataFrame()
count_all = 100 #number of reviews for each app 
for row_app in crypto_app.itertuples():
    #download reviews from App Store
    app_store = AppStore(country = 'us', app_name = row_app[2], app_id = row_app[3])
    app_store.review(how_many = 50)      #get reviews
    df = pd.DataFrame(app_store.reviews)
    #merge title and review into 'review' 
    df['review'] = df['title'] + '. ' + df['review']
    #leave only the required columns
    df = df.loc[:, ['rating', 'review', 'date', 'userName']]
    df['app_name'] = row_app[1]   #App name
    df['marketplace'] = 'Apple App Store' 
    df_all = df_all.append(df)    #add data to general frame
    
    #download reviews from GooglePlay
    #determine the right amount 
    count_for_gp = count_all - df.shape[0]
    google_play, continuation_token = reviews(row_app[4], count = count_for_gp)   #get reviews
    df2 = pd.DataFrame(google_play)
    #leave only the required columns
    df2 = df2.loc[:, ['content', 'score', 'at', 'userName']]
    #rename columns for uniformity
    df2.rename(columns = {'content':'review', 'score':'rating', 'at':'date'}, inplace = True)
    df2['app_name'] = row_app[1]   #App name
    df2['marketplace'] = 'Google Play Store'
    df_all = df_all.append(df2)    #add data to general frame

2023-01-25 13:23:15,935 [INFO] Base - Initialised: AppStore('us', 'crypto-com-buy-bitcoin-eth', 1262148500)
2023-01-25 13:23:15,936 [INFO] Base - Ready to fetch reviews from: https://apps.apple.com/us/app/crypto-com-buy-bitcoin-eth/id1262148500
2023-01-25 13:23:16,389 [INFO] Base - [id:1262148500] Fetched 60 reviews (60 fetched in total)
2023-01-25 13:23:16,945 [INFO] Base - Initialised: AppStore('us', 'binance-buy-bitcoin-crypto', 1436799971)
2023-01-25 13:23:16,945 [INFO] Base - Ready to fetch reviews from: https://apps.apple.com/us/app/binance-buy-bitcoin-crypto/id1436799971
2023-01-25 13:23:17,403 [INFO] Base - [id:1436799971] Fetched 60 reviews (60 fetched in total)
2023-01-25 13:23:18,015 [INFO] Base - Initialised: AppStore('us', 'kucoin-buy-bitcoin-ether', 1378956601)
2023-01-25 13:23:18,018 [INFO] Base - Ready to fetch reviews from: https://apps.apple.com/us/app/kucoin-buy-bitcoin-ether/id1378956601
2023-01-25 13:23:18,532 [INFO] Base - [id:1378956601] Fetched 60 reviews (60 fe

In [4]:
df_all['country'] = 'us' #add country
#rename columns
df_all.rename(columns = {'review':'app_review_text', 'date':'timestamp'}, inplace = True)

In [5]:
#see how many rows and columns
df_all.shape

(1000, 7)

In [6]:
#see how many are in the Apple App Store and Google Play Store
df_all.marketplace.value_counts()

Apple App Store      526
Google Play Store    474
Name: marketplace, dtype: int64

In [7]:
#look at the collected data
df_all.head()

,rating,app_review_text,timestamp,userName,app_name,marketplace,country
0,4,"Great app. So far I am very new to this app, h...",2021-08-25 19:03:14,$cashguy$,Crypto.com,Apple App Store,us
1,5,Wonderful Crypto App Review Rating. I was so d...,2022-12-09 20:29:00,Joann172,Crypto.com,Apple App Store,us
2,5,Definitely sufficient wonderful app review. I ...,2023-01-13 19:40:15,Theodore$2G4,Crypto.com,Apple App Store,us
3,5,Excellent trading service app. I was so dissat...,2022-12-22 22:09:16,KayT07839$,Crypto.com,Apple App Store,us
4,5,A great crypto app 💯. I was impatient to carry...,2022-11-29 02:23:55,Maria Bronce 🍀🥰🍀,Crypto.com,Apple App Store,us


In [8]:
df_all.describe()

,rating
count,1000.000000
mean,3.735000
std,1.670798
min,1.000000
25%,2.000000
50%,5.000000
75%,5.000000
max,5.000000


In [9]:
#save data to file
df_all.to_csv('Reviews_CryptoApp.csv', index = False)